In [19]:
import re
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


import nltk

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 120
from sklearn import preprocessing


In [24]:
!unzip -q //content/drive/MyDrive/datasets/IMDBDataset.csv.zip
!ls


replace IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:  drive	'IMDB Dataset.csv'   sample_data


In [26]:
df = pd.read_csv('/content/IMDB Dataset.csv', encoding='latin-1')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [27]:
def clean(text):
    text = text.lower() #привод к нижнему регистру
    text = re.sub(r'http\S+', " ", text) #
    text = re.sub(r'@\w+',' ',text) #
    text = re.sub(r'#\w+', ' ', text) #
    text = re.sub(r'\d+', ' ', text) #
    text = re.sub(r'<.*?>',' ', text) #
    return text

In [28]:
len(df) ,df.isnull().sum()

(50000, review       0
 sentiment    0
 dtype: int64)

In [29]:
le = preprocessing.LabelEncoder()
le.fit_transform(df['review'])

array([29427,  3472, 19496, ..., 12485, 21422, 27736])

In [30]:
text = df['review']
text.head()

0    One of the other reviewers has mentioned that ...
1    A wonderful little production. <br /><br />The...
2    I thought this was a wonderful way to spend ti...
3    Basically there's a family where a little boy ...
4    Petter Mattei's "Love in the Time of Money" is...
Name: review, dtype: object

In [31]:
text = text.apply(clean)
text.head(15)

0     one of the other reviewers has mentioned that ...
1     a wonderful little production.   the filming t...
2     i thought this was a wonderful way to spend ti...
3     basically there's a family where a little boy ...
4     petter mattei's "love in the time of money" is...
5     probably my all-time favorite movie, a story o...
6     i sure would like to see a resurrection of a u...
7     this show was an amazing, fresh & innovative i...
8     encouraged by the positive comments about this...
9     if you like original gut wrenching laughter yo...
10    phil the alien is one of those quirky films wh...
11    i saw this movie when i was about   when it ca...
12    so im not a big fan of boll's work but then ag...
13    the cast played shakespeare.  shakespeare lost...
14    this a fantastic movie of three prisoners who ...
Name: review, dtype: object

In [32]:
from nltk.corpus import stopwords

In [33]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

In [34]:
wn_lemmatizer = WordNetLemmatizer()

In [35]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [36]:
dirty_tweet = text.iloc[520]

In [37]:
print(f'''
      ** Original Tweet **: \n \n      
      {dirty_tweet}
      
      ------------------------------------------------
      
      ** WordNetLemmatizer: ** \n \n
      {' '.join([wn_lemmatizer.lemmatize(word) for word in dirty_tweet.split()])}
      ''')


      ** Original Tweet **: 
 
      
      i am a huge fan of the farcry game, huge fan. it still holds a place in my top-  games list of all time! the story line was new, fresh... a truly brilliant foundation to base a movie on... or so i thought...  farcry the movie is no less than another directors attempt at cashing in on a successful game franchise (see doom: the movie, and many more...).  the video game begins as the player (jack carver) awakes in a sea side cave after been shot off his boat by an rpg from an unknown soldier. jack then finds a communication device where (harlan doyle) guides him across islands, shipwrecks, jungles, installations and volcanoes, to find his (lady friend?) (valerie constantine), all the wile battling mutated super soldiers, and genetically enhanced animals.  the movie plays out very, very differently:  : there's a needless   mins ( /  of the movie) of "backstory" before we even get to the 'boat blowing up' scene.  : jack then walks onto the beach,

In [38]:
lemmatized_text = []
for tweet in text:
    lemmatized_text.append(' '.join([wn_lemmatizer.lemmatize(word) for word in tweet.split()]))

In [39]:
reg_tokenizer = RegexpTokenizer('\w+')

In [40]:
tokenized_text = reg_tokenizer.tokenize_sents(lemmatized_text)

In [41]:
len(tokenized_text)

50000

In [42]:
  >>> import nltk
  >>> nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [43]:
sw = stopwords.words()

In [44]:
clean_tokenized_tweets = [] 
for i, element in enumerate(tokenized_text):
    if i % 2000 == 0: print(i, end = ' ')
    clean_tokenized_tweets.append(' '.join([word for word in element if word not in sw]))

0 2000 4000 6000 8000 10000 12000 14000 16000 18000 20000 22000 24000 26000 28000 30000 32000 34000 36000 38000 40000 42000 44000 46000 48000 

In [45]:
df = pd.concat([pd.Series(clean_tokenized_tweets, name='review'), 
                pd.Series(df['sentiment'], name='sentiment')], 
               axis=1)

In [46]:
df['review'][3]

'basically family little boy jake think zombie closet parent fighting time movie slower soap opera suddenly jake decides become rambo kill zombie ok first going make film must decide thriller drama drama movie watchable parent divorcing arguing like real life jake closet totally ruin film expected see boogeyman similar movie instead watched drama meaningless thriller spots well playing parent descent dialogs shot jake ignore'

In [47]:
df.to_csv('clean_review', index=False)

`CountVectorizer`: make **bag of words** representation 

`TfidfVectorizer`: make **frequency based** representation

In [48]:
# Create objects
cvec = CountVectorizer(ngram_range=(1, 2))
tfid = TfidfVectorizer(ngram_range=(1, 2))

In [49]:
cvec_representation = cvec.fit_transform(pd.Series(clean_tokenized_tweets))
tfid_representation = tfid.fit_transform(pd.Series(clean_tokenized_tweets))

In [50]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier

In [51]:
mnb = MultinomialNB()
lrc = LogisticRegression()
svc = SVC()
ls=Lasso()
sgd=SGDClassifier()
rf=RandomForestClassifier()
sgd=SGDClassifier

In [55]:
clfs = {
    'MultiNB' : mnb, 
   # 'LogisticRegression' : lrc,
     
   #  'SGDClassifier': sgd

}

In [56]:
def get_scores(clfs, n_cv=2):
    for clf in clfs: 
        cvec_scores = cross_val_score(clfs[clf], cvec_representation, df['sentiment'], n_jobs=4, cv=n_cv, scoring='accuracy')
        tfidf_scores = cross_val_score(clfs[clf], tfid_representation,  df['sentiment'], n_jobs=4, cv=n_cv, scoring='accuracy')

        cvec_mean_score, cvec_std_score = np.mean(cvec_scores), np.std(cvec_scores)
        tfidf_mean_score, tfidf_std_score = np.mean(tfidf_scores), np.std(tfidf_scores)
        print(f''' 
        {clf}
        CountVectorizer score: {cvec_mean_score:.3f}, std: {cvec_std_score:.3f}
        TfIdf score: {tfidf_mean_score:.3f}, std: {tfidf_std_score:.3f}
        ''')
        

In [57]:
get_scores(clfs, 2)

 
        MultiNB
        CountVectorizer score: 0.879, std: 0.000
        TfIdf score: 0.882, std: 0.002
        


# Prediction

In [ ]:
df = pd.read_csv('/content/IMDB Dataset.csv', encoding='latin-1')
df.head()

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from gensim.models import Word2Vec
from gensim.parsing.preprocessing import STOPWORDS # more stopwords in gensim corpus
 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
import re

from sklearn.manifold import TSNE

stop_nltk = stopwords.words('english') # nltk corpus

STOPWORDS = STOPWORDS.union(set(["don't", "i'm"]))

In [ ]:
y = df['sentiment'].replace({'Negative' : 0, 
                                 'Positive' : 1})

In [ ]:
reg_tok = RegexpTokenizer('\w+')

In [ ]:
def clean_and_lemmatize(text):
    #cleaning
    text = text.lower()
    text = re.sub(r'http\S+', " ", text)
    text = re.sub(r'@\w+',' ',text)
    text = re.sub(r'#\w+', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub('r<.*?>',' ', text)
    text = reg_tok.tokenize(text)
    
    
    # filtering
    text = " ".join([word for word in text if not word in STOPWORDS and len(word) > 2])
    
    # lemmatization
    text = WordNetLemmatizer().lemmatize(text)
    text = text.split()
    
    
    return text

In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
preprocessed_tweets = df['review'].apply(clean_and_lemmatize)
preprocessed_tweets

In [ ]:
len(preprocessed_tweets)

In [ ]:
wv = Word2Vec(min_count=20,
              window=3,
              size=400, 
              workers=-1)

In [ ]:
import gensim
gensim.__version__

In [ ]:
wv.build_vocab(preprocessed_tweets)

In [ ]:
# train model
wv.train(preprocessed_tweets, 
         total_examples=len(preprocessed_tweets), 
         epochs=20, 
         queue_factor=2)

In [ ]:
X = wv[wv.wv.vocab]
words = list(wv.wv.vocab)

In [ ]:
X_embed = TSNE(n_iter=500).fit_transform(X)

In [ ]:
plt.figure(figsize=(18, 8))
plt.scatter(X_embed[:60, 0], X_embed[:60, 1])
annotations = []
for i, word in enumerate(words[:60]):
    annotations.append(plt.annotate(word, xy=(X_embed[i, 0]+.05, X_embed[i, 1]), fontsize=9, alpha=.5, ))
adjust_text(annotations)